**Import libraries**

In [12]:
import math
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

**Default parameters**

In [13]:
MAX_SIZE = 783
NUM_WORDS = 1000
EMBEDDING_DIM = 16
EPOCHS = 10
BATCH_SIZE = 512
OOV = 0
SARCASM_TRAINING_SIZE = 20000

**Creating DataFrames**

In [14]:
goodreads_train = pd.read_csv("kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv")
goodreads_test = pd.read_csv("kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv")

In [15]:
goodreads_train.sample(5)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
389797,0924fe07d528643b74fb5010623527b9,1656001,a7c047bb684e612ce8b56a7c2fbea5d0,1,Nomadic SA Chick's Book Reviews \n Summary \n ...,Tue Jan 13 23:30:48 -0800 2015,Sat Oct 24 13:45:27 -0700 2015,Thu Jul 09 00:00:00 -0700 2015,Tue Jun 30 00:00:00 -0700 2015,1,0
250240,3e1f81789c6e74ea2267b90b393e07cb,30312891,e6f19ae6f74c454d321d2a6aa57015c9,4,"Really, Clare, really? You're going to do that...",Sat Jul 18 15:03:54 -0700 2015,Sun Jul 09 10:56:33 -0700 2017,Sun Jul 09 14:56:42 -0700 2017,Sat Jul 08 09:03:46 -0700 2017,6,2
95288,909e430324d1d07372797a539e9c114f,224500,72942a788f8b9300248b9fb14c32bcdd,4,I'm not sure why Goodreads seems to have lost ...,Wed Jul 30 12:52:30 -0700 2008,Thu Oct 01 13:18:58 -0700 2015,Fri Nov 03 00:00:00 -0800 2006,NaN,0,0
279750,b6c07403a8d278f4f80bf73f5fae038d,129645,47f6511cab47a2b34a7363975794e55a,4,"Loved the story .. The three friends , their f...",Sat Jan 31 05:13:28 -0800 2015,Sat Dec 31 08:46:54 -0800 2016,Sat Dec 31 08:46:54 -0800 2016,Sat Dec 31 00:00:00 -0800 2016,6,0
683640,6ace7ac40603395904fcf04d0156f64f,15746882,cdfcdf324a993e63e86b89aebabcf561,3,"This is kind of bittersweet for me, to be hone...",Thu Oct 03 00:59:49 -0700 2013,Mon Jul 17 06:04:52 -0700 2017,Mon Jul 17 06:04:52 -0700 2017,Wed Jun 28 00:00:00 -0700 2017,0,0


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
261328,c2b309a38e1b723e8eacbd3e14bbc61b,852470,2caffedbd71473eed1a18586ecad9edb,3,"This book is a YA romance, spy-style. Pretty m...",Fri Nov 28 12:26:53 -0800 2008,Fri Nov 28 12:28:35 -0800 2008,Thu Nov 27 00:00:00 -0800 2008,NaN,2,0
705873,b29da45436a66d3837267b7466159c2c,23447923,9f3cea50e65e9f95caa38f62a5d631f8,5,"A beautiful slice-of-life novel about a boy, a...",Thu Mar 02 15:50:30 -0800 2017,Thu Mar 02 15:52:18 -0800 2017,Thu Mar 02 15:52:18 -0800 2017,Mon Feb 20 00:00:00 -0800 2017,0,0
725431,22d913a77b4e7596a382acf520f10746,3398625,cc2979411beed02f921bd10dab99314d,5,This was a superbly written book. Focusing on ...,Tue Jan 04 21:26:09 -0800 2011,Fri Jan 21 21:57:03 -0800 2011,Fri Jan 21 00:00:00 -0800 2011,Thu Jan 20 00:00:00 -0800 2011,0,2
728187,ffa5094acb2bca8fc8655538e60c400e,17455811,dd651868a5fec856d531c5f72a4db033,3,Was fun. I have a sinking feeling I'm going to...,Tue Jul 09 14:45:54 -0700 2013,Tue Dec 10 23:10:50 -0800 2013,Tue Dec 10 00:00:00 -0800 2013,NaN,0,0
524401,3ab3421e36cba34c77602255b3b35c15,12063467,365b9ceea302304409222ab663b6a7cc,4,I really enjoyed this book but not as much as ...,Fri Jun 03 18:37:40 -0700 2016,Sun Jun 05 06:08:43 -0700 2016,Sun Jun 05 06:08:43 -0700 2016,Sat Jun 04 00:00:00 -0700 2016,0,0


In [16]:
goodreads_test.sample(5)

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
124130,a63a6cb4a04e72c647077c2019053f20,41536,861c826a938431a6e349a4ca4a902297,I would give it more of a 4 1/2 star rating bu...,Wed Sep 14 14:18:47 -0700 2011,Fri Nov 18 22:04:17 -0800 2011,Fri Nov 18 00:00:00 -0800 2011,Wed Nov 09 00:00:00 -0800 2011,0,0
434955,a35fb741fd524fe86626d0528711c0b4,18812405,5a90141f53ee82a685f35693d91b2b3d,There are not enough words for me to describe ...,Sat Sep 19 10:59:25 -0700 2015,Mon Sep 18 21:58:11 -0700 2017,Mon Sep 18 21:34:41 -0700 2017,Sun Sep 17 13:04:53 -0700 2017,2,0
19585,8539029f322064f0785f4a22944b05ed,22152502,33d7c9d07e7d5f116ee735e4fc2df265,"After the disappointment that was Dark 28, I w...",Fri Dec 23 05:05:55 -0800 2016,Fri Dec 23 05:24:21 -0800 2016,NaN,NaN,0,0
295933,f8f67f1c3512b41f715dabcedbb062a8,9378297,b6827e5fbc6550c75343fcb84eb09248,I so want to read this.,Fri Feb 24 16:48:56 -0800 2012,Fri Feb 24 16:49:12 -0800 2012,NaN,NaN,0,13
142126,3044d39612948e06d8394d92748c36e8,12877181,a91b7b803e4d9a847f643d742e59f269,This book is more of the same as the other thr...,Sun Oct 16 21:02:34 -0700 2011,Thu Mar 07 20:17:40 -0800 2013,Sun Oct 23 00:00:00 -0700 2011,Sun Oct 16 00:00:00 -0700 2011,1,0


,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
210993,5a8d502e5eb75d8383e1c9b57b4739ac,12510082,432c25db80cab0d44cb5559a81297aba,Collins has done it again with a briliant nove...,Sat Aug 27 16:56:20 -0700 2011,Thu May 05 07:16:51 -0700 2016,Fri Mar 23 00:00:00 -0700 2012,Tue Mar 20 00:00:00 -0700 2012,1,0
85625,c91960168fde66b8b5afc59be169dd92,18712886,e06ca534a191e978c059b5e7792d2396,"T H O U G H T S: \n a lackluster, unexceptiona...",Fri Aug 12 12:58:15 -0700 2016,Mon Oct 17 18:35:52 -0700 2016,Fri Aug 12 00:00:00 -0700 2016,Fri Aug 12 00:00:00 -0700 2016,0,0
204641,3ff83680d9087b5394b6844731ac15a3,13576132,44f8da2538e07475d70b561e09a518e4,3.5 \n when will we have half stars?? when????,Mon Apr 20 19:40:19 -0700 2015,Fri May 29 15:21:14 -0700 2015,Fri May 29 15:21:14 -0700 2015,Fri May 29 00:00:00 -0700 2015,0,2
374405,1180a110d2bd28f9353da90159784d73,6063187,7bb584637233c66971fe75904179c170,This was an interesting read. A mix of chick-l...,Sat Feb 25 03:03:23 -0800 2012,Fri Mar 16 00:16:32 -0700 2012,NaN,NaN,0,0
260712,3037e45de740da82703e55a19f94cfbe,8685612,0e3fc2a38a10511c7a207f036029695c,I thought The Iron King was good \n Iron Daugh...,Wed Oct 12 23:43:37 -0700 2011,Thu Apr 19 00:46:45 -0700 2012,Thu Apr 19 00:46:45 -0700 2012,Tue Apr 17 23:49:21 -0700 2012,0,0


**Cleaning Data**

In [17]:
train_df = goodreads_train.drop(columns=['user_id', 'book_id', 'date_added', 'date_updated', 'read_at', 'started_at'],
                                axis=0)
train_df.head()

,review_id,rating,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",9,1


,review_id,rating,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",9,1


In [18]:
train_df.dtypes

review_id      object
rating          int64
review_text    object
n_votes         int64
n_comments      int64
dtype: object

review_id      object
rating          int64
review_text    object
n_votes         int64
n_comments      int64
dtype: object

In [19]:
y_train = train_df['rating']
x_train = train_df.drop('rating', axis=1)

In [20]:
x_train.head()

,review_id,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,"I really enjoyed this book, and there is a lot...",9,1


,review_id,review_text,n_votes,n_comments
0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,This is a special book. It started slow for ab...,28,1
1,a5d2c3628987712d0e05c4f90798eb67,Recommended by Don Katz. Avail for free in Dec...,1,0
2,2ede853b14dc4583f96cf5d120af636f,"A fun, fast paced science fiction thriller. I ...",22,0
3,ced5675e55cd9d38a524743f5c40996e,Recommended reading to understand what is goin...,5,1
4,332732725863131279a8e345b63ac33e,"I really enjoyed this book, and there is a lot...",9,1


In [21]:
y_train.head()

0    5
1    3
2    3
3    0
4    4
Name: rating, dtype: int64

0    5
1    3
2    3
3    0
4    4
Name: rating, dtype: int64

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [23]:
x_train['review_id'] = le.fit_transform(x_train['review_id'])

In [13]:
x_train.head()

,review_id,review_text,n_votes,n_comments
0,786842,This is a special book. It started slow for ab...,28,1
1,583423,Recommended by Don Katz. Avail for free in Dec...,1,0
2,165147,"A fun, fast paced science fiction thriller. I ...",22,0
3,727692,Recommended reading to understand what is goin...,5,1
4,179941,"I really enjoyed this book, and there is a lot...",9,1


**NLP**

In [14]:
tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token=OOV)

In [15]:
def get_sequences(tokenizer, review):
    sequences = tokenizer.texts_to_sequences(review)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=MAX_SIZE, padding='post')
    return padded_sequences

In [16]:
def tokenizer_func(data_rating, data_review):
    tokenizer.fit_on_texts(data_review)

    train_labels = data_rating.iloc[math.floor(int(len(data_rating) / 8)):]
    train_examples = data_review.iloc[math.floor(int(len(data_review) / 8)):]
    test_examples = data_review.iloc[:math.floor(int(len(data_review) / 8))]
    test_labels = data_rating.iloc[:math.floor(int(len(data_rating) / 8))]

    padded_train = get_sequences(tokenizer, train_examples)
    padded_test = get_sequences(tokenizer, test_examples)

    return np.array(padded_train), np.array(padded_test), np.array(train_labels), np.array(test_labels)

In [17]:
padded_train, padded_test, train_labels, test_labels = tokenizer_func(y_train, x_train['review_text'])

In [18]:
padded_train[0]

array([  4,  12, 294,  16,   6,   1, 566,  31,   3,   9,  10, 456,  42,
         4, 151,  88, 597,   1,  59,  10,   6,   1,   7,   6,   1,   1,
        21,  10, 635,  16,   2,   1,   1, 459,  45,  34, 185,   1,  27,
         6, 428,   5,   1,   2, 223,   7,   2, 888,   1,   1, 388,  42,
        80,   4, 134,  47,   1, 362,   3, 566,  32,  42,   4,  12, 294,
        16,   4,  91,  37, 291,  13,  14,  15,   9, 167, 209, 124, 207,
        25, 132,  48, 156, 290,   3,  84,   1, 255, 111, 383, 242, 124,
         3,  42, 180,   7, 460,  10,   1,   1,  24,   2, 434,  70, 914,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

**Sarcasm detection**

In [19]:
data = pd.read_json('kaggle/input/sarcasmjson/sarcasm.json', lines=True)

In [20]:
# Iterating through the json data and loading the requisite values into our python lists
sentences = data['headline']
labels = data['is_sarcastic']

In [21]:
training_sentences = sentences[0:SARCASM_TRAINING_SIZE]
testing_sentences = sentences[SARCASM_TRAINING_SIZE:]

In [22]:
training_labels = labels[0:SARCASM_TRAINING_SIZE]
testing_labels = labels[SARCASM_TRAINING_SIZE:]

In [23]:
tokenizer.fit_on_texts(training_sentences)

In [24]:
# Creating training sequences and padding them
training_padded = get_sequences(tokenizer, training_sentences)
testing_padded = get_sequences(tokenizer, testing_sentences)

In [25]:
# Converting all variables to numpy arrays, to be able to work with tf version 2
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [26]:
# Creating a model for sentiment analysis
sarcasm_model = tf.keras.Sequential([
    # Adding an Embedding layer for Neural Network to learn the vectors
    tf.keras.layers.Embedding(NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SIZE),
    # Global Average pooling is similar to adding up vectors in this case
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [27]:
sarcasm_model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

In [28]:
sarcasm_model.fit(training_padded, training_labels, epochs=EPOCHS,
                    validation_data=(testing_padded, testing_labels))

Epoch 1/10
625/625 [==============================] - 6s 6ms/step - loss: 0.6864 - accuracy: 0.5585 - val_loss: 0.6847 - val_accuracy: 0.5633
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6847 - accuracy: 0.5603 - val_loss: 0.6835 - val_accuracy: 0.5633
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.6807 - accuracy: 0.5603 - val_loss: 0.6742 - val_accuracy: 0.5633
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6571 - accuracy: 0.5738 - val_loss: 0.6328 - val_accuracy: 0.5840
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5999 - accuracy: 0.6775 - val_loss: 0.5869 - val_accuracy: 0.6369
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5433 - accuracy: 0.7300 - val_loss: 0.5192 - val_accuracy: 0.7450
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5016 - accuracy: 0.7557 - val_loss: 0.4986 - val_accuracy: 0.7511
Epoch 

In [29]:
sarcasm_prediction_train = sarcasm_model.predict(padded_train)
sarcasm_prediction_test = sarcasm_model.predict(padded_test)

3516/3516 [==============================] - 4s 1ms/step


**Reshaping data**


In [30]:
sarcasm_prediction_train.shape

(787500, 1)

In [31]:
padded_train.shape

(787500, 783)

In [32]:
padded_train = np.concatenate((padded_train, np.array(sarcasm_prediction_train.flatten())[:, None]), axis=1)
padded_test = np.concatenate((padded_test, np.array(sarcasm_prediction_test.flatten())[:, None]), axis=1)

In [33]:
padded_train.shape

(787500, 784)

In [34]:
padded_train = np.reshape(padded_train, (1 - math.floor(len(y_train) / 8), 28, 28))
padded_test = np.reshape(padded_test, (math.floor(int(len(x_train['review_text']) / 8)), 28, 28))

In [35]:
padded_train[0]

array([[4.0000000e+00, 1.2000000e+01, 2.9400000e+02, 1.6000000e+01,
        6.0000000e+00, 1.0000000e+00, 5.6600000e+02, 3.1000000e+01,
        3.0000000e+00, 9.0000000e+00, 1.0000000e+01, 4.5600000e+02,
        4.2000000e+01, 4.0000000e+00, 1.5100000e+02, 8.8000000e+01,
        5.9700000e+02, 1.0000000e+00, 5.9000000e+01, 1.0000000e+01,
        6.0000000e+00, 1.0000000e+00, 7.0000000e+00, 6.0000000e+00,
        1.0000000e+00, 1.0000000e+00, 2.1000000e+01, 1.0000000e+01],
       [6.3500000e+02, 1.6000000e+01, 2.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 4.5900000e+02, 4.5000000e+01, 3.4000000e+01,
        1.8500000e+02, 1.0000000e+00, 2.7000000e+01, 6.0000000e+00,
        4.2800000e+02, 5.0000000e+00, 1.0000000e+00, 2.0000000e+00,
        2.2300000e+02, 7.0000000e+00, 2.0000000e+00, 8.8800000e+02,
        1.0000000e+00, 1.0000000e+00, 3.8800000e+02, 4.2000000e+01,
        8.0000000e+01, 4.0000000e+00, 1.3400000e+02, 4.7000000e+01],
       [1.0000000e+00, 3.6200000e+02, 3.000000

**CNN Training**

In [36]:
cnn_model = tf.keras.models.Sequential()

In [37]:
cnn_model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(8, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.MaxPool2D())

cnn_model.add(tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.MaxPool2D())

cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.tanh, padding='same'))
cnn_model.add(tf.keras.layers.MaxPool2D())

cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(tf.keras.layers.Dense(32, activation=tf.keras.activations.relu))  # tf.keras.activations.tanh
cnn_model.add(tf.keras.layers.Dense(16, activation=tf.keras.activations.relu))  # tf.keras.activations.tanh
cnn_model.add(tf.keras.layers.Dense(6,
                                    activation=tf.keras.activations.softmax))  # model.add(tf.keras.layers.Dense(1, activation=tf.keras.activations.softmax))

In [38]:
cnn_model.compile(optimizer=tf.keras.optimizers.SGD(0.1, momentum=0.9),
                  loss=tf.keras.losses.categorical_crossentropy,
                  metrics=[tf.keras.metrics.categorical_accuracy])

In [39]:
cnn_model.fit(padded_train,
              train_labels,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS
              )

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

**Testing model**

In [ ]:
padded_train = padded_train / MAX_SIZE
padded_test = padded_test / MAX_SIZE

train_labels = tf.keras.utils.to_categorical(train_labels, 6)
test_labels = tf.keras.utils.to_categorical(test_labels, 6)

padded_train = np.expand_dims(padded_train, -1)
padded_test = np.expand_dims(padded_test, -1)

cnn_predict = cnn_model.predict(padded_train)

In [ ]:
cnn_model.summary()

**Submission**

In [ ]:
sample_submission = pd.read_csv("kaggle/input/goodreads-books-reviews-290312/goodreads_sample_submission.csv")
predictions = np.argmax(cnn_predict, axis=1)
sample_submission.rating = predictions
sample_submission.to_csv("kaggle/working/submission.csv", index=False)